# Vacancy and Substitutional defect pair in GaN

Focusing on recreating the defect pair in GaN from the Figure 1. c) in the publication:

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect_point_pair_gallium_nitride/0-figure-from-manuscript.webp" width="400" />

## 1. Prepare the Environment
### 1.1. Set up defects parameters 
Defect Configuration parameters are described in [Defect Configuration](https://github.com/Exabyte-io/made/blob/8196b759242551c77d1791bf5bd2f4150763cfef/src/py/mat3ra/made/tools/build/defect/configuration.py#L102).

In [ ]:
from types import SimpleNamespace

SUPERCELL_MATRIX = [[3, 0, 0], [0, 3, 0], [0, 0, 2]]

# Configuration using SimpleNamespace for dot notation access
PRIMARY_DEFECT_CONFIG = SimpleNamespace(
    defect_type="substitution",
    coordinate=[0.5, 0.5, 0.5],
    element="Mg",
    placement_method="closest_site",
)

SECONDARY_DEFECT_CONFIG = SimpleNamespace(
    defect_type="vacancy",
    coordinate=[0.5, 0.5, 0.65],
    placement_method="closest_site",
)

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+g4d0dde0-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_point_defect.ipynb")

### 1.3. Get input materials
Materials are loaded with `get_materials()`.

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material
from utils.visualize import visualize_materials as visualize

material = Material.create(Materials.get_by_name_first_match("GaN"))
visualize(material)

In [ ]:
unit_cell = material


### 1.4. Create and preview Supercell

In [ ]:
from mat3ra.made.tools.build.supercell import create_supercell

supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="-90x")

In [ ]:
from mat3ra.made.tools.build.defect.pair_defect.helpers import create_pair_defect

material_with_defect = create_pair_defect(
    material=supercell,
    defect_type_1=PRIMARY_DEFECT_CONFIG.defect_type,
    coordinate_1=PRIMARY_DEFECT_CONFIG.coordinate,
    element_1=PRIMARY_DEFECT_CONFIG.element if hasattr(PRIMARY_DEFECT_CONFIG, "element") else None,
    placement_method_1=PRIMARY_DEFECT_CONFIG.placement_method if hasattr(PRIMARY_DEFECT_CONFIG, 'placement_method') else None,
    defect_type_2=SECONDARY_DEFECT_CONFIG.defect_type,
    coordinate_2=SECONDARY_DEFECT_CONFIG.coordinate,
    element_2=SECONDARY_DEFECT_CONFIG.element if hasattr(SECONDARY_DEFECT_CONFIG, "element") else None,
    placement_method_2=SECONDARY_DEFECT_CONFIG.placement_method if hasattr(SECONDARY_DEFECT_CONFIG, 'placement_method') else None,
)


## 2. Create the Defect

## 3. Visualize Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([{"material": supercell, "title": "Original material"},
           {"material": material_with_defect, "title": f"Material with defect"}],
          viewer="wave")

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(material_with_defect)
download_content_to_file(material_with_defect.to_json(), "Mg substitution and vacancy in GaN.json")